### Packages Required

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from keras.models import Model
from keras.layers import Input,LSTM,Dense

In [22]:
batch_size = 64
epochs = 30
latent_dim = 256
num_samples = 10000
data_path = ('C:/Users/Nithin/Downloads/Machine Translation/Datasets/DATASET.txt')

In [4]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [5]:
with open(data_path,"r",encoding = "utf-8") as f:
    lines = f.read().split("\n")

In [7]:
for line in lines[:min(num_samples,len(lines)-1)]:
    input_text,target_text = line.split("\t")
    target_text = "\t"+ target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [8]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [9]:
print('no of samples : ',len(input_texts))
print('no of unique input words :',num_encoder_tokens)
print('no of unique Output words :',num_decoder_tokens)
print('max seq length for inputs :',max_encoder_seq_length)
print('max seq length for inputs :',max_decoder_seq_length)

no of samples :  2867
no of unique input words : 70
no of unique Output words : 92
max seq length for inputs : 124
max seq length for inputs : 123


In [10]:
input_token_index = dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i,char in enumerate(target_characters)])

In [11]:
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype = 'float32'
)

In [12]:
decoder_input_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype = 'float32'
)

In [13]:
decoder_target_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype = 'float32'
)

In [14]:
for i, (input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]] = 1.0
        encoder_input_data[i,t + 1 :,input_token_index[" "]] = 1.0
    for t,char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i,t-1,target_token_index[char]] = 1.0
            
    decoder_input_data[i,t + 1 :,target_token_index[" "]] = 1.0
    decoder_target_data[i,t:,target_token_index[" "]] = 1.0

In [15]:
len(decoder_target_data[decoder_target_data==1])

352641

In [16]:
len(decoder_input_data[decoder_input_data==1])

352641

In [17]:
encoder_input_data.shape

(2867, 124, 70)

In [18]:
decoder_input_data.shape

(2867, 123, 92)

In [19]:
decoder_target_data.shape

(2867, 123, 92)

In [20]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

M0 = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [23]:
M0.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics = ['accuracy'])

M0.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)

Epoch 1/30
36/36 [==============================] - 97s 2s/step - loss: 0.7901 - accuracy: 0.8076 - val_loss: 1.3294 - val_accuracy: 0.6857
Epoch 2/30
36/36 [==============================] - 63s 2s/step - loss: 0.7608 - accuracy: 0.8105 - val_loss: 1.2590 - val_accuracy: 0.6876
Epoch 3/30
36/36 [==============================] - 59s 2s/step - loss: 0.7660 - accuracy: 0.8131 - val_loss: 1.3011 - val_accuracy: 0.6938
Epoch 4/30
36/36 [==============================] - 57s 2s/step - loss: 0.7271 - accuracy: 0.8161 - val_loss: 1.2748 - val_accuracy: 0.6845
Epoch 5/30
36/36 [==============================] - 61s 2s/step - loss: 0.7213 - accuracy: 0.8213 - val_loss: 1.1932 - val_accuracy: 0.7073
Epoch 6/30
36/36 [==============================] - 62s 2s/step - loss: 0.6811 - accuracy: 0.8302 - val_loss: 1.1644 - val_accuracy: 0.7092
Epoch 7/30
36/36 [==============================] - 89s 2s/step - loss: 0.6665 - accuracy: 0.8361 - val_loss: 1.1054 - val_accuracy: 0.7256
Epoch 8/30
36/36 [==

In [24]:
M0.save('C:/Users/Nithin/Downloads/Machine Translation/Models/Machine Transalation.h5')

In [25]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [26]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

In [27]:
for seq_index in range(10):
    
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print(input_texts[seq_index])
    print(decoded_sentence)

1/1 [==============================] - 0s 34ms/step
-
Wow!
मैं कर सक में को से को को को को किया।

1/1 [==============================] - 0s 34ms/step
-
Help!
मैं कर सक में को से को को को को किया।

1/1 [==============================] - 0s 17ms/step
-
Jump.
मैं कर सक में को से को को को को किया।

1/1 [==============================] - 0s 34ms/step
-
Jump.
मैं कर सक में को से को को को को किया।

1/1 [==============================] - 0s 16ms/step
-
Jump.
मैं कर सक में को से को को को को किया।

1/1 [==============================] - 0s 18ms/step
-
Hello!
मैं कर सक में को से को को को को किया।

1/1 [==============================] - 0s 17ms/step
-
Hello!
मैं कर सक में को से को को को को किया।

1/1 [==============================] - 0s 34ms/step
-
Cheers!
मैं कर सक में को से को को को को किया।

1/1 [==============================] - 0s 32ms/step
-
Cheers!
मैं कर सक में को से को को को को किया।

1/1 [==============================] - 0s 21ms/step
-
Got it?
मैं कर सक में को से को को को को किया।

